
##### Extra : Random Forest 

Dataset disponibles :
- X_train : avec valeurs Nan , sans transformation (Hyperplot+300 -> 0.677)
- X_train_forest_imputed : Nan imputés avec Forest (Hyperplot+300 -> 0.673)
- X__Mice_imputed : Nan imputés avec Mice  (Hyperplot+300 -> 0.672)
- X_poly_2_No_nan : Nan imputés avec Forest + polynomes et interractions  (Hyperplot+300 -> 0.657)
- X_train_RandomUnderSampler , y_train_RandomUnderSampler : données parfaitement équilibrées mais moins importantes. (Hyperplot+300 -> 0.65) 
- X_train_del : imputation will bring a lot of noise (Hyperplot+300 -> 0.672, 1000 -> 0.673)
- X_train_poly2_outnan : polynomes sans OP100 (nan values) (Hyperplot+300 -> 0.657)

Les tests nous montrent que quelque soit la méthode d'imputation retenue , il y a perte de performances. Ceci est très probablement du  au fait que la création de bruit par les méthodes d'imputation.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Créer un objet Optuna Study
study = create_study(direction="maximize")

class_weights = {0: 1, 1: 1}

# Définition de l'espace de recherche
def objective(trial: Trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 400)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    min_weight_fraction_leaf = trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5)


    # Créer un modèle RandomForestClassifier avec les paramètres suggérés
    clf = RandomForestClassifier(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  min_samples_split=min_samples_split,
                                  min_samples_leaf=min_samples_leaf,
                                  min_weight_fraction_leaf= min_weight_fraction_leaf,
                                  class_weight = class_weights, 
                                  n_jobs = -1)
    
    # Évaluation à l'aide de la validation croisée avec ROC AUC comme métrique
    scores = cross_val_score(clf, X_train_del, y_train, cv=5, scoring='roc_auc')
    return scores.mean()

# Création de l'étude Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

# Affichage des meilleurs hyperparamètres et de la meilleure ROC AUC moyenne
best_params = study.best_params
best_auc = study.best_value

[I 2023-10-29 11:27:22,447] A new study created in memory with name: no-name-263f6c98-9898-4e57-bef1-24e8853c6a28
[I 2023-10-29 11:27:22,452] A new study created in memory with name: no-name-a7c178f9-3ae5-4624-8f61-7f0f464b9b8d
[I 2023-10-29 11:27:25,712] Trial 0 finished with value: 0.6267820261547529 and parameters: {'n_estimators': 233, 'max_depth': 1, 'min_samples_split': 2, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.21067862744674337}. Best is trial 0 with value: 0.6267820261547529.
[I 2023-10-29 11:27:26,531] Trial 1 finished with value: 0.64363574067596 and parameters: {'n_estimators': 17, 'max_depth': 2, 'min_samples_split': 10, 'min_samples_leaf': 6, 'min_weight_fraction_leaf': 0.07365139001072918}. Best is trial 1 with value: 0.64363574067596.
[I 2023-10-29 11:27:29,052] Trial 2 finished with value: 0.6199371289192596 and parameters: {'n_estimators': 278, 'max_depth': 7, 'min_samples_split': 2, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.34068296395225006}.

KeyboardInterrupt: 

In [ ]:
best_auc

0.669919158907615

In [ ]:
optuna.visualization.plot_optimization_history(study)